In [1]:
import json
import jieba
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.externals import joblib
from sklearn import preprocessing
import numpy as np

import TfIdf_SinglePass

## 测试4k个feature

### 得到tf-idf

In [7]:
def get_tf_idf_of_rumor(features_num=4000):
    corpus = []
    with open('file/corpus/cut_corpus_of_rumor.txt', 'r') as src:
        lines = src.readlines()
        for line in lines:
            corpus.append(line)
        print('The size of corpus is {}'.format(len(corpus)))

    vectorizer = CountVectorizer(max_features=features_num)
    transformer = TfidfTransformer()
    tf_idf = transformer.fit_transform(vectorizer.fit_transform(corpus))
    vocabulary = vectorizer.get_feature_names()

    joblib.dump((vocabulary, tf_idf), 'file/pkl/tf_idf_of_rumor_4000.pkl')

get_tf_idf_of_rumor()

The size of corpus is 34611


In [10]:
vocabulary, tf_idf = joblib.load('file/pkl/tf_idf_of_{}.pkl'.format('rumor_4000'))
tf_idf_array = preprocessing.normalize(tf_idf.toarray(), norm='l2')

In [12]:
# vocabulary[]

In [23]:
tf_idf_array.shape

(34611, 4000)

### 查看聚类效果

In [18]:
def _clustering(category, threshold=0.6):
    _, tf_idf = joblib.load('file/pkl/tf_idf_of_{}.pkl'.format(category))
    tf_idf_array = preprocessing.normalize(tf_idf.toarray(), norm='l2')

    single_pass_cluster = TfIdf_SinglePass.SinglePassCluster(tf_idf_array[:1000], t=threshold)
    joblib.dump(single_pass_cluster, 'file/pkl/tf_idf_{}_clustering.pkl'.format(category))

In [22]:
# _clustering('rumor_4000')

In [20]:
def test_on_client(category):
    single_pass_cluster = joblib.load('file/pkl/tf_idf_{}_clustering.pkl'.format(category))
    cluster_list = single_pass_cluster.cluster_list
    with open('file/corpus/corpus_of_rumor.txt', 'r') as src:
        with open('file/test_{}.txt'.format(category), 'w') as out:
            lines = src.readlines()
            for cluster in cluster_list:
                for i in cluster.node_list:
                    out.write('{}'.format(lines[i]))
                out.write('-----------------------------------\n')

In [21]:
test_on_client('rumor_4000')